<a href="https://colab.research.google.com/github/rkdwodms/Data-analysis/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/gdrive')

# 데이터 불러오기

train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data/Titanic_train.csv')
test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/data/Titanic_test.csv')


Mounted at /gdrive


In [2]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, random_state=2)

train.shape, test.shape

((668, 13), (418, 12))

In [3]:
#타겟 범주 비율 확인
target = 'Survived'

y_train = train[target]
y_train.value_counts(normalize = True)

0    0.625749
1    0.374251
Name: Survived, dtype: float64

In [4]:
#기준모델 : 최빈값

basemd = y_train.mode()[0]
y_pred = [basemd] * len(y_train)

In [6]:
#정확도 구하기 
from sklearn.metrics import accuracy_score
print("training accuracy: ", accuracy_score(y_train, y_pred))

y_val = val[target]
y_pred = [basemd] * len(y_val)
print("validation accuracy: ", accuracy_score(y_val, y_pred))

training accuracy:  0.625748502994012
validation accuracy:  0.5874439461883408


In [8]:
#로지스틱 회귀로 문제 풀기
train.head()
train = train.set_index(train.columns[0])

In [9]:
train['Ticket'].value_counts()
# Ticket 특성 제거 (high cardinality)
# 생존에 영향을 주지 않는 특성 제거 후 모델링

347088          6
382652          5
S.O.C. 14879    5
3101295         5
CA. 2343        5
               ..
W./C. 6609      1
A/5. 2151       1
PC 17600        1
349206          1
W./C. 14263     1
Name: Ticket, Length: 539, dtype: int64

In [10]:
!pip install --upgrade category_encoders

     |████████████████████████████████| 80 kB 4.9 MB/s 


In [11]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
# 범주형 변수 one hot encoding 수행
encoder = OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_train_encoded.head(3)

# 결측치 평균으로 변환
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train_encoded)
X_val_imputed = imputer.transform(X_val_encoded)

# 특성값 표준화 : 특성들의 척도를 맞추기 위함 / 평균으로부터 얼마나 떨어져 있는지 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

#평균은 0 표준편차는 1 

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [19]:
X_train_encoded, X_train_imputed , X_train_scaled

(            Pclass  Sex_female  Sex_male  ...  Embarked_Q  Embarked_C  Embarked_nan
 Unnamed: 0                                ...                                      
 199              2           1         0  ...           0           0             0
 129              3           0         1  ...           0           0             0
 90               3           0         1  ...           0           0             0
 230              1           1         0  ...           0           0             0
 126              3           0         1  ...           1           0             0
 ...            ...         ...       ...  ...         ...         ...           ...
 534              3           1         0  ...           0           0             0
 584              3           0         1  ...           0           1             0
 493              1           0         1  ...           0           1             0
 527              1           0         1  ...           0       

In [18]:
model = LogisticRegression(random_state=1)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_val_scaled)
accuracy_score(y_val, y_pred)

#78%정도의 정확도

0.7892376681614349

In [20]:
# 특성별 계수로 영향도 확인 
coeff = pd.Series(model.coef_[0], X_train_encoded.columns)
coeff

# 좌석등급 숫자가 작을 수록, 여성일수록, 나이가 어릴수록 생존이 높아짐.

Pclass         -0.915833
Sex_female      0.662095
Sex_male       -0.662095
Age            -0.559957
SibSp          -0.406466
Parch          -0.015897
Fare            0.078016
Embarked_S     -0.094939
Embarked_Q      0.007684
Embarked_C      0.077224
Embarked_nan    0.188837
dtype: float64